In [64]:
import matplotlib
matplotlib.use('TkAgg')  
import matplotlib.pyplot as plt
import pandas as pd
import scikit_posthocs as sp
from imblearn.over_sampling import SMOTE
from collections import Counter
from sklearn.preprocessing import OrdinalEncoder
import seaborn as sns
import scipy.stats as stats
from random import randint, uniform
from scipy.stats import randint as sp_randint, uniform
import warnings
import numpy as np 
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
import joblib  
import os  
import plotly.express as px



# --- Variable Encoding Function ---
def encode_variables(df):  # Changed function name to English
    """
    Encodes the DataFrame variables according to the specified plan.

    Args:
        df: pandas DataFrame with the original data.

    Returns:
        DataFrame: DataFrame with encoded variables.
    """

    df_encoded = df.copy()

    # 1. anxiety_level: Ordinal Encoding (0, 1, 2, 3)
    anxiety_mapping = {
        "minimal": 0,
        "mild": 1,
        "moderate": 2,
        "severe": 3,
    }
    df_encoded["anxiety_level"] = df_encoded["anxiety_level"].map(anxiety_mapping)

    # 2. sex: Binary Encoding (0 and 1)
    sex_mapping = {"female": 0, "male": 1}
    df_encoded["sex"] = df_encoded["sex"].map(sex_mapping)

    # 3. education_level: Ordinal Encoding (0, 1, 2)
    education_mapping = {
        "technical level": 0,
        "bachelor": 1,
        "graduate": 2,
    }
    df_encoded["education_level"] = df_encoded["education_level"].map(
        education_mapping
    )

    # 4. shift: One-Hot Encoding
    df_encoded = pd.get_dummies(
        df_encoded, columns=["shift"], prefix="shift", dummy_na=False
    )

    # 5. marital_status: One-Hot Encoding + Handling 'widowed'
    df_encoded["marital_status"] = df_encoded["marital_status"].replace(
        "widowed", "single"
    )  # Group with 'single'
    df_encoded = pd.get_dummies(
        df_encoded, columns=["marital_status"], prefix="marital", dummy_na=False
    )

    # 6. category: One-Hot Encoding + Grouping
    df_encoded["category"] = df_encoded["category"].replace(
        ["spec nurse", "head nurse"], "other"
    )  # Group
    df_encoded = pd.get_dummies(
        df_encoded, columns=["category"], prefix="category", dummy_na=False
    )

    # 7. age_range: Ordinal Encoding
    age_categories = ["20 to 29", "30 to 39", "40 to 49", "50 and over"]
    age_encoder = OrdinalEncoder(categories=[age_categories])
    df_encoded["age_range"] = age_encoder.fit_transform(
        df_encoded[["age_range"]]
    )
    df_encoded["age_range"] = df_encoded["age_range"].astype(int)

    # 8. seniority_range: Ordinal Encoding
    seniority_categories = [
        "1 to 5",
        "6 to 10",
        "11 to 15",
        "16 to 20",
        "21 and over",
    ]
    seniority_encoder = OrdinalEncoder(categories=[seniority_categories])
    df_encoded["seniority_range"] = seniority_encoder.fit_transform(
        df_encoded[["seniority_range"]]
    )
    df_encoded["seniority_range"] = df_encoded["seniority_range"].astype(int)

    return df_encoded


# --- Data Augmentation Function with SMOTE ---
def augment_data_smote(
    df, target_variable, sampling_strategy="auto", random_state=None, k_neighbors=None
):
    """
    Applies SMOTE to oversample the target variable, with improved error handling
    and k_neighbors adjustment.

    Args:
        df: pandas DataFrame with the encoded data.
        target_variable: Name of the column containing the (categorical) target variable.
        sampling_strategy: Sampling strategy.
        random_state: Random seed.
        k_neighbors: Number of neighbors.

    Returns:
        DataFrame, Series: DataFrame with augmented predictor variables and Series with the
                         augmented target variable.  Or None, None if there's an error.
    """

    X = df.drop(target_variable, axis=1)
    y = df[target_variable]

    if not pd.api.types.is_numeric_dtype(y):
        print("Error: The target variable must be numeric (ordinally encoded).")
        return None, None

    if len(y.unique()) < 2:
        print("Error: The target variable must have at least two different classes.")
        return None, None

    class_counts = Counter(y)
    min_samples = min(class_counts.values())

    if k_neighbors is None:
        k_neighbors = min(5, min_samples - 1)
    else:
        k_neighbors = min(k_neighbors, min_samples - 1)

    if k_neighbors < 1:
        print("Error: At least one class has very few samples. SMOTE cannot be applied.")
        print("Consider grouping minority classes or removing the solitary sample.")
        return None, None

    if min_samples <= 1:
        print("Error: At least one class has only one sample. SMOTE cannot be applied.")
        print("Consider grouping minority classes or removing the solitary sample before applying SMOTE")
        return None, None

    print(f"Using k_neighbors = {k_neighbors} for SMOTE.")

    try:
        smote = SMOTE(
            sampling_strategy=sampling_strategy,
            random_state=random_state,
            k_neighbors=k_neighbors,
        )
        X_resampled, y_resampled = smote.fit_resample(X, y)
    except ValueError as e:
        print(f"Error applying SMOTE: {e}")
        return None, None

    print("Class distribution before SMOTE:", Counter(y))
    print("Class distribution after SMOTE:", Counter(y_resampled))

    return X_resampled, y_resampled


def analyze_spearman_correlation(df, target_variable):
    """
    Calculates and visualizes Spearman correlations between the (ordinal) target variable
    and other numerical/ordinal variables in the DataFrame.

    Args:
        df: pandas DataFrame with the data (already augmented and encoded).
        target_variable: Name of the column containing the (ordinal) target variable.

    Returns:
        None (prints the correlation matrix and shows a heatmap).
    """

    # Calculate the Spearman correlation matrix
    spearman_correlations = df.corr(method="spearman")

    # Extract correlations with the target variable
    correlations_with_target = spearman_correlations[target_variable].drop(
        target_variable
    )  # Exclude self-correlation

    # Print correlations with the target variable
    print("Spearman Correlations with", target_variable, ":\n")
    print(correlations_with_target.sort_values(ascending=False))

    # Visualize with a heatmap (all variables)
    plt.figure(figsize=(12, 10))
    sns.heatmap(spearman_correlations, annot=True, cmap="coolwarm", center=0)
    plt.title("Spearman Correlation Heatmap")
    plt.tight_layout()
    plt.savefig("spearman_correlation_heatmap.png")  # Save the figure
    plt.close() #Close figure

    # Visualize with a bar plot (only correlations with the target variable)
    plt.figure(figsize=(8, 6))
    correlations_with_target.sort_values().plot(kind="barh", color="skyblue")
    plt.title(f"Spearman Correlation with {target_variable}")
    plt.xlabel("Spearman Correlation Coefficient")
    plt.ylabel("Variables")
    plt.tight_layout()
    plt.savefig("spearman_correlation_barplot.png")  # Save the figure
    plt.close()


def inferential_analysis(df, target_variable):
    """
    Performs Kruskal-Wallis and Chi-square tests to analyze relationships
    between the target variable and other variables in the DataFrame.

    Args:
        df: pandas DataFrame with the data (already augmented and encoded).
        target_variable: Name of the column containing the (ordinal) target variable.

    Returns:
        None (prints the test results).
    """

    # --- 1. Kruskal-Wallis (for numerical/ordinal variables) ---
    print("-" * 50)
    print("KRUSKAL-WALLIS TESTS")
    print("-" * 50)

    # List of numerical/ordinal variables (excluding target and one-hot encoded)
    numerical_ordinal_vars = [
        col
        for col in df.columns
        if (
            pd.api.types.is_numeric_dtype(df[col])
            and col != target_variable
            and not col.startswith(("shift_", "marital_", "category_"))
        )
    ]

    for variable in numerical_ordinal_vars:
        # Group data by levels of the target variable
        groups = [
            df[variable][df[target_variable] == level]
            for level in df[target_variable].unique()
        ]

        # Check for at least two groups and non-empty groups
        if len(groups) < 2:
            print(
                f"Cannot perform Kruskal-Wallis on {variable}: fewer than two groups."
            )
            continue
        if any(len(group) == 0 for group in groups):
            print(
                f"Cannot perform Kruskal-Wallis on {variable}: at least one group is empty."
            )
            continue

        # Perform Kruskal-Wallis test
        try:
            statistic, p_value = stats.kruskal(*groups)
        except ValueError as e:
            print(f"Error performing Kruskal-Wallis on {variable}: {e}")
            print("This may occur if all values in a group are equal.")
            continue

        print(f"\nKruskal-Wallis: {variable} vs. {target_variable}")
        print(f"  H Statistic: {statistic:.3f}")
        print(f"  p-value: {p_value:.3f}")

        # Interpretation
        if p_value < 0.05:
            print(
                f"  Result: Significant differences between levels of {target_variable} on variable {variable}."
            )

            # --- Post-Hoc Tests (Dunn with Bonferroni/Holm correction) ---
            # Dunn's test
            dunn_result = sp.posthoc_dunn(
                a=df,
                val_col=variable,
                group_col=target_variable,
                p_adjust="bonferroni",  # Or 'holm'
            )
            print("\n  Dunn's Test (Post-Hoc):")
            print(dunn_result)

        else:
            print(
                f"  Result: No significant differences between levels of {target_variable} on variable {variable}."
            )

    # --- 2. Chi-square (for categorical variables) ---
    print("\n" + "-" * 50)
    print("CHI-SQUARE TESTS")
    print("-" * 50)

    # List of categorical variables (including one-hot encoded)
    categorical_vars = [
        col
        for col in df.columns
        if col
        in (
            "sex",
            "shift_afternoon",
            "shift_morning",
            "shift_night a",
            "shift_night b",
            "marital_domestic partnership",
            "marital_married",
            "marital_single",
            "category_gen nurse",
            "category_nurse aux",
            "category_other",
            "education_level"
        )
    ]

    for variable in categorical_vars:
        # Create contingency table
        contingency_table = pd.crosstab(df[target_variable], df[variable])
        print(f"\nContingency Table: {target_variable} vs. {variable}")
        print(contingency_table)

        # Perform Chi-square test
        try:
            chi2, p_value, dof, expected = stats.chi2_contingency(contingency_table)
            print(f"\nChi-square: {variable} vs. {target_variable}")
            print(f"  Chi2 Statistic: {chi2:.3f}")
            print(f"  p-value: {p_value:.3f}")
            print(f"  Degrees of freedom: {dof}")

            # Interpretation
            if p_value < 0.05:
                print(
                    f"  Result: Significant association between {target_variable} and {variable}."
                )
                # Calculate Cramer's V
                n = contingency_table.sum().sum()
                phi2 = chi2 / n
                r, k = contingency_table.shape
                phi2corr = max(0, phi2 - ((k - 1) * (r - 1)) / (n - 1))
                rcorr = r - ((r - 1) ** 2) / (n - 1)
                kcorr = k - ((k - 1) ** 2) / (n - 1)
                cramers_v = (phi2corr / min((kcorr - 1), (rcorr - 1))) ** 0.5
                print(f"  Cramer's V: {cramers_v:.3f}")

            else:
                print(
                    f"  Result: No significant association between {target_variable} and {variable}."
                )

        except ValueError as e:
            print(f"Error performing Chi-square on {variable}: {e}")
            print("This may occur if any expected frequencies are zero.")
            continue

def train_model(model, X_train, y_train):
    """Trains a given model."""
    model.fit(X_train, y_train)
    return model

def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluates a model and prints metrics.  Returns test accuracy."""
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    conf_matrix = confusion_matrix(y_test, model.predict(X_test))

    # Handle multiclass classification report
    try:
        class_report = classification_report(y_test, model.predict(X_test))
    except ValueError as e:
        print(f"Error generating classification report: {e}")
        print("This usually happens if the predicted labels don't cover all classes present in y_test.")
        print("Check if your model is predicting all classes, especially after SMOTE and train/test split.")
        class_report = "Could not generate report (see error above)"

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}\n")
    print(f"Confusion Matrix:\n{conf_matrix}\n")
    print(f"Classification Report:\n{class_report}")
    return test_accuracy



def evaluate_model(model, X_train, y_train, X_test, y_test):
    """Evaluates a model and prints metrics.  Returns test accuracy."""
    train_accuracy = accuracy_score(y_train, model.predict(X_train))
    test_accuracy = accuracy_score(y_test, model.predict(X_test))
    conf_matrix = confusion_matrix(y_test, model.predict(X_test))

    # Handle multiclass classification report
    try:
        class_report = classification_report(y_test, model.predict(X_test))
    except ValueError as e:
        print(f"Error generating classification report: {e}")
        print("This usually happens if the predicted labels don't cover all classes present in y_test.")
        print("Check if your model is predicting all classes, especially after SMOTE and train/test split.")
        class_report = "Could not generate report (see error above)"

    print(f"Training Accuracy: {train_accuracy:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}\n")
    print(f"Confusion Matrix:\n{conf_matrix}\n")
    print(f"Classification Report:\n{class_report}")
    return test_accuracy



def optimize_and_evaluate(model_name, model, param_dist, X_train, y_train, X_test, y_test, n_iter=20, cv=5):
    """Performs RandomizedSearchCV, saves/loads results, evaluates, and returns test accuracy."""

    save_path = f"{model_name}_random_search.pkl"  # Simplified path

    try:
        if os.path.exists(save_path):
            random_search = joblib.load(save_path)
            print(f"Loaded RandomizedSearchCV for {model_name} from {save_path}")
        else:
            print(f"No checkpoint found for {model_name}. Starting RandomizedSearchCV.")
            print(f"Type of param_dist: {type(param_dist)}")  # Debug: Check type
            print(f"Contents of param_dist: {param_dist}") # Debug: Check contents

            random_search = RandomizedSearchCV(
                model,
                param_dist,
                n_iter=n_iter,
                cv=cv,
                scoring='accuracy',
                n_jobs=-1,
                random_state=42,
                verbose=0
            )
            random_search.fit(X_train, y_train)
            joblib.dump(random_search, save_path)  # Save after fitting
            print(f"RandomizedSearchCV for {model_name} saved to {save_path}")


        best_model = random_search.best_estimator_
        print(f"Best parameters for {model_name}: {random_search.best_params_}")
        test_accuracy = evaluate_model(best_model, X_train, y_train, X_test, y_test)
        return test_accuracy

    except Exception as e:
        print(f"Error in optimize_and_evaluate for {model_name}: {e}")
        return None


def create_models_dict():
    """Creates the dictionary of models with pipelines for scaling."""
    models = {
        "KNN": Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier())]),
        "Decision Tree": DecisionTreeClassifier(random_state=42),
        "Random Forest": RandomForestClassifier(random_state=42),
        "AdaBoost": AdaBoostClassifier(random_state=42),
        "Gradient Boosting": Pipeline([('scaler', StandardScaler()), ('gb', GradientBoostingClassifier(random_state=42))]),
        "XGBoost": Pipeline([('scaler', StandardScaler()), ('xgb', XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='mlogloss'))]), #Multiclass classification
        "Extra Trees": ExtraTreesClassifier(random_state=42),
    }
    return models


def create_param_distributions():
    """Defines parameter distributions for RandomizedSearchCV."""
    param_distributions = {
        "KNN": {
            'knn__n_neighbors': sp_randint(3, 21),  # Use sp_randint
            'knn__weights': ['uniform', 'distance'],
            'knn__p': [1, 2]
        },
        "Decision Tree": {
            'max_depth': sp_randint(3, 21),  # Use sp_randint
            'min_samples_split': sp_randint(2, 21),
            'min_samples_leaf': sp_randint(1, 21),
            'criterion': ['gini', 'entropy', "log_loss"]
        },
        "Random Forest": {
            'n_estimators': sp_randint(100, 501),  # Use sp_randint
            'max_depth': sp_randint(3, 21),
            'min_samples_split': sp_randint(2, 21),
            'min_samples_leaf': sp_randint(1, 21),
            'max_features': [None, 'sqrt', 'log2'],
            'bootstrap': [True, False]
        },
        "AdaBoost": {
            'n_estimators': sp_randint(50, 201),  # Use sp_randint
            'learning_rate': uniform(0.01, 0.5)
        },
        "Gradient Boosting": {
            'gb__n_estimators': sp_randint(100, 501),  # Use sp_randint
            'gb__learning_rate': uniform(0.01, 0.2),
            'gb__max_depth': sp_randint(3, 11),
            'gb__min_samples_split': sp_randint(2, 11),
            'gb__min_samples_leaf': sp_randint(1, 11),
            'gb__subsample': uniform(0.7, 0.3),
            'gb__max_features': [None, 'sqrt', 'log2']
        },
        "XGBoost": {
            'xgb__n_estimators': sp_randint(100, 501),  # Use sp_randint
            'xgb__learning_rate': uniform(0.01, 0.2),
            'xgb__max_depth': sp_randint(3, 11),
            'xgb__subsample': uniform(0.7, 0.3),
            'xgb__colsample_bytree': uniform(0.7, 0.3),
            'xgb__gamma': uniform(0, 0.5)
        },
        "Extra Trees": {
            'n_estimators': sp_randint(100, 501),  # Use sp_randint
            'max_depth': sp_randint(3, 21),
            'min_samples_split': sp_randint(2, 21),
            'min_samples_leaf': sp_randint(1, 21),
            'max_features': [None, 'sqrt', 'log2'],
            'bootstrap': [True, False]
        }
    }
    return param_distributions

In [65]:
# --- Main Flow ---

# 1. Load Data
df = pd.read_csv(r"D:\ansiedad\AnxietyLevelByCovid.csv")

In [66]:
# 2. Encode Variables
df_encoded = encode_variables(df)


In [67]:
# 3. Augment Data
sampling_strategy = {
    0: 100,  # minimal
    1: 100,  # mild
    2: 100,  # moderate
    3: 100,  # severe
}

X_resampled, y_resampled = augment_data_smote(
    df_encoded,
    target_variable="anxiety_level",
    sampling_strategy=sampling_strategy,
    random_state=42,
    #k_neighbors=3,  # Optional
)

Using k_neighbors = 5 for SMOTE.
Class distribution before SMOTE: Counter({0: 62, 1: 49, 2: 22, 3: 7})
Class distribution after SMOTE: Counter({1: 100, 0: 100, 2: 100, 3: 100})


In [68]:
# 4. Create Augmented DataFrame
if X_resampled is not None and y_resampled is not None:
    df_augmented = pd.DataFrame(X_resampled, columns=X_resampled.columns)
    df_augmented["anxiety_level"] = y_resampled

    # --- Now work with df_augmented ---
    print("\nAugmented DataFrame:")
    print(df_augmented.head())
    print(df_augmented.shape)

    # --- Perform correlation analysis, inferential analysis, etc. ---
    analyze_spearman_correlation(df_augmented, "anxiety_level")
    inferential_analysis(df_augmented, "anxiety_level")


else:
    print("Data augmentation failed. Check for errors.")


Augmented DataFrame:
   sex  education_level  age_range  seniority_range  shift_afternoon  \
0    0                1          0                0            False   
1    0                2          2                3            False   
2    1                2          1                2            False   
3    0                1          2                3            False   
4    0                2          1                2            False   

   shift_morning  shift_night a  shift_night b  marital_domestic partnership  \
0          False           True          False                          True   
1           True          False          False                         False   
2           True          False          False                         False   
3           True          False          False                          True   
4           True          False          False                         False   

   marital_married  marital_single  category_gen nurse  category

In [69]:
df_augmented.shape

(400, 15)

In [70]:
analyze_spearman_correlation(df_augmented, "anxiety_level")

Spearman Correlations with anxiety_level :

education_level                 0.085992
category_other                 -0.065167
marital_single                 -0.067109
shift_night a                  -0.090665
shift_night b                  -0.116411
category_gen nurse             -0.144150
shift_afternoon                -0.155972
shift_morning                  -0.160083
sex                            -0.170128
marital_domestic partnership   -0.182868
marital_married                -0.198373
age_range                      -0.208170
category_nurse aux             -0.214688
seniority_range                -0.267693
Name: anxiety_level, dtype: float64


## Spearman Correlation Analysis Summary

This section summarizes the findings from the Spearman rank correlation analysis, focusing on the relationships between the `anxiety_level` (our ordinal target variable) and other predictor variables in the dataset (after data augmentation with SMOTE).

**Key Findings:**

*   **Generally Weak Correlations:**  Overall, most predictor variables exhibit weak correlations with `anxiety_level`, with no Spearman correlation coefficients exceeding |0.3|. This suggests that anxiety, as measured in this study, is likely influenced by a combination of factors, rather than any single strong predictor.
* **Negative Correlations:**
    *  `seniority_range` and `age_range` show the strongest negative correlation, indicating a mild tendency.
    *   `category_other` (representing specialist and head nurses) shows a notable negative correlation (-0.21) with `anxiety_level`.  This suggests that nurses in these roles tend to report lower anxiety levels compared to general and auxiliary nurses.
    *   `education_level` exhibits a weak negative correlation (-0.14), suggesting a slight tendency for nurses with higher education levels to report lower anxiety.
*   **Positive Correlations:**
    *   `category_nurse_aux` shows the strongest positive correlation, indicating a mild tendency.
    * `sex` exhibits a very weak possitive correlation.
*   **Multicollinearity:**  A strong positive correlation (0.69) exists between `age_range` and `seniority_range`.  This indicates multicollinearity, which should be addressed before using these variables together in regression models.  Consider using only one of these variables or creating a composite variable.
* **Variables with very weak correlation**
    *  `shift_afternoon`, `shift_morning`, `marital_married`, `shift_night a`, `shift_night b`, `marital_domestic partnership`, `marital_single`, `category_gen nurse`.

**Next Steps:**

These correlation findings provide a valuable starting point for further investigation.  The next steps involve:

1.  **Hypothesis Testing:** Conducting non-parametric tests (Kruskal-Wallis for numerical/ordinal predictors, Chi-square for categorical predictors) to formally test the statistical significance of the observed associations.
2.  **Visualization:** Creating visualizations (box plots, stacked bar plots) to explore the relationships between `anxiety_level` and other variables in more detail.
3.  **Predictive Modeling:** Building classification models (e.g., ordinal logistic regression, decision trees) to predict `anxiety_level`, while carefully addressing the multicollinearity issue and considering potential interactions between variables.

**Important Note:** Correlation does not imply causation.  These findings highlight associations, but further research is needed to understand the causal mechanisms underlying anxiety levels in nurses.

In [71]:
inferential_analysis(df_augmented, "anxiety_level")

--------------------------------------------------
KRUSKAL-WALLIS TESTS
--------------------------------------------------

Kruskal-Wallis: sex vs. anxiety_level
  H Statistic: 31.240
  p-value: 0.000
  Result: Significant differences between levels of anxiety_level on variable sex.

  Dunn's Test (Post-Hoc):
          0         1         2         3
0  1.000000  0.026143  0.290413  0.290413
1  0.026143  1.000000  0.000008  0.000008
2  0.290413  0.000008  1.000000  1.000000
3  0.290413  0.000008  1.000000  1.000000

Kruskal-Wallis: education_level vs. anxiety_level
  H Statistic: 3.744
  p-value: 0.290
  Result: No significant differences between levels of anxiety_level on variable education_level.

Kruskal-Wallis: age_range vs. anxiety_level
  H Statistic: 25.195
  p-value: 0.000
  Result: Significant differences between levels of anxiety_level on variable age_range.

  Dunn's Test (Post-Hoc):
         0         1         2         3
0  1.00000  1.000000  1.000000  0.000140
1  1.00000

## Statistical Analysis: Key Conclusions

This section summarizes the key findings from the statistical analysis, building upon the exploratory data analysis and correlation analysis.  We used non-parametric hypothesis tests (Kruskal-Wallis and Chi-square) to assess the statistical significance of relationships between the ordinal `anxiety_level` variable and other predictors.

**Main Conclusions:**

*   **Significant Associations with Anxiety:** Several demographic and work-related factors show statistically significant associations with nurses' anxiety levels during the COVID-19 pandemic (p < 0.05, after appropriate corrections for multiple comparisons).  These factors include:
    *   **Sex:**  There is a significant association between sex and anxiety levels, as confirmed by both the Kruskal-Wallis and Chi-square tests. Cramer's V suggests a moderate association strength.
    *   **Age Range (`age_range`):**  Kruskal-Wallis revealed significant differences in anxiety levels across age groups. Post-hoc Dunn tests indicated that the "severe" anxiety group had a significantly different age distribution compared to the other anxiety levels.
    *   **Seniority Range (`seniority_range`):**  Similar to age, seniority shows significant differences in anxiety levels across groups, confirmed by Kruskal-Wallis and Dunn's post-hoc tests (with the "severe" group differing significantly).
    *   **Shift (`shift_afternoon`, `shift_morning`, `shift_night a`, `shift_night b`):**  Chi-square tests showed significant associations between anxiety levels and *all* shift types.  This indicates that the distribution of anxiety levels differs across the various shifts.
    *   **Marital Status (`marital_domestic partnership`, `marital_married`, `marital_single`):** Chi-square tests revealed significant associations between anxiety levels and all marital status categories.
    *   **Nurse Category (`category_gen nurse`, `category_nurse aux`, `category_other`):**  Chi-square tests showed significant associations between anxiety and all nurse category types.  This reinforces the importance of the nurse's role in their anxiety experience.

*   **Non-Significant Association with Education Level:**  `education_level` did *not* show a statistically significant association with anxiety levels in either the Kruskal-Wallis or Chi-square tests.  This suggests that, in this sample, education level alone is not a strong predictor of anxiety.

* **Strength of Associations:**  While several associations were statistically significant, it's important to remember that the correlation analysis (Spearman) generally showed weak correlations.  The Chi-square tests, while showing significance, also had Cramer's V values that were mostly in the low to moderate range. This reinforces the idea that anxiety is likely influenced by multiple factors, each with a relatively small to moderate individual effect.


In [72]:
if __name__ == "__main__":

    # 1. Load Data
 # 2. Prepare data for modeling (train/test split)
    X = df_augmented.drop('anxiety_level', axis=1)
    y = df_augmented['anxiety_level']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y)  # Stratify!

    # 3. Create models and parameter distributions
    models = create_models_dict()
    param_distributions = create_param_distributions()


    # --- Model Optimization and Evaluation Loop ---
    results = {}
    for name, model in models.items():
        print(f"--- Optimizing and Evaluating {name} ---")
        if name in param_distributions:  # Only optimize if we have defined distributions
            test_accuracy = optimize_and_evaluate(name, model, param_distributions[name], X_train, y_train, X_test, y_test)
            if test_accuracy is not None:
                results[name] = test_accuracy
        else:  # Train without optimization if no distributions
            print(f"No parameter distributions found for {name}.  Training without optimization.")
            trained_model = train_model(model, X_train, y_train)
            test_accuracy = evaluate_model(trained_model, X_train, y_train, X_test, y_test)
            results[name] = test_accuracy

    # --- Model Comparison (with Plotly) ---
    print("\n--- Model Comparison ---")
    results_df = pd.DataFrame(list(results.items()), columns=['Model', 'Test Accuracy'])
    results_df = results_df.sort_values(by='Test Accuracy', ascending=False)  # Sort before plotting
    print(results_df)

    # Create the bar chart with Plotly
    fig = px.bar(results_df, x='Test Accuracy', y='Model',
                  title='Model Comparison (Test Accuracy)',
                  color='Test Accuracy',  # Color by accuracy
                  color_continuous_scale=px.colors.sequential.Viridis,  # Color scale
                  template='plotly_dark',  # Use a dark theme (optional)
                  text='Test Accuracy')  # Show accuracy on bars

    fig.update_traces(texttemplate='%{text:.4f}', textposition='outside')  # Format text
    fig.update_layout(xaxis_title='Accuracy', yaxis_title='Model', xaxis_range=[0, 1])  # Titles and x-axis range
    fig.show()

--- Optimizing and Evaluating KNN ---
No checkpoint found for KNN. Starting RandomizedSearchCV.
Type of param_dist: <class 'dict'>
Contents of param_dist: {'knn__n_neighbors': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000025A35E37A10>, 'knn__weights': ['uniform', 'distance'], 'knn__p': [1, 2]}
RandomizedSearchCV for KNN saved to KNN_random_search.pkl
Best parameters for KNN: {'knn__n_neighbors': 13, 'knn__p': 1, 'knn__weights': 'distance'}
Training Accuracy: 0.9143
Test Accuracy: 0.6833

Confusion Matrix:
[[23  4  2  1]
 [ 9 15  5  1]
 [ 6  3 20  1]
 [ 3  0  3 24]]

Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.77      0.65        30
           1       0.68      0.50      0.58        30
           2       0.67      0.67      0.67        30
           3       0.89      0.80      0.84        30

    accuracy                           0.68       120
   macro avg       0.70      0.68      0.68       120

C:\Users\Manuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning:

The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.



RandomizedSearchCV for AdaBoost saved to AdaBoost_random_search.pkl
Best parameters for AdaBoost: {'learning_rate': np.float64(0.20993048585762775), 'n_estimators': 64}
Training Accuracy: 0.5536
Test Accuracy: 0.5333

Confusion Matrix:
[[11  7  7  5]
 [ 3 13  9  5]
 [ 3  8 13  6]
 [ 2  1  0 27]]

Classification Report:
              precision    recall  f1-score   support

           0       0.58      0.37      0.45        30
           1       0.45      0.43      0.44        30
           2       0.45      0.43      0.44        30
           3       0.63      0.90      0.74        30

    accuracy                           0.53       120
   macro avg       0.53      0.53      0.52       120
weighted avg       0.53      0.53      0.52       120

--- Optimizing and Evaluating Gradient Boosting ---
No checkpoint found for Gradient Boosting. Starting RandomizedSearchCV.
Type of param_dist: <class 'dict'>
Contents of param_dist: {'gb__n_estimators': <scipy.stats._distn_infrastructure.rv_di

C:\Users\Manuel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\xgboost\core.py:158: UserWarning:

[20:09:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.




RandomizedSearchCV for XGBoost saved to XGBoost_random_search.pkl
Best parameters for XGBoost: {'xgb__colsample_bytree': np.float64(0.9131988669057362), 'xgb__gamma': np.float64(0.05544541040591566), 'xgb__learning_rate': np.float64(0.09786730037315403), 'xgb__max_depth': 5, 'xgb__n_estimators': 162, 'xgb__subsample': np.float64(0.9687290787020557)}
Training Accuracy: 0.8821
Test Accuracy: 0.7083

Confusion Matrix:
[[24  4  2  0]
 [ 8 17  3  2]
 [ 3  6 17  4]
 [ 2  0  1 27]]

Classification Report:
              precision    recall  f1-score   support

           0       0.65      0.80      0.72        30
           1       0.63      0.57      0.60        30
           2       0.74      0.57      0.64        30
           3       0.82      0.90      0.86        30

    accuracy                           0.71       120
   macro avg       0.71      0.71      0.70       120
weighted avg       0.71      0.71      0.70       120

--- Optimizing and Evaluating Extra Trees ---
No checkpoint f